<a href="https://colab.research.google.com/github/siji3328/OligoDesign/blob/main/barcodecombi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import pandas as pd

# 엑셀 파일 업로드
uploaded = files.upload()  # 사용자로부터 파일 업로드
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")

# 업로드된 파일에서 B 열 읽고 FASTA 파일로 변환
input_file = list(uploaded.keys())[0]  # 업로드된 첫 번째 파일 사용
output_file = "output_sequences.fasta"

# 엑셀 파일 읽기, header=None으로 설정하여 첫 번째 행을 헤더로 사용하지 않음
df = pd.read_excel(input_file, header=None)

# B 열에서 데이터 가져오기 (이제 열 인덱스 1을 사용)
# 0부터 시작하는 인덱스를 사용하여 두 번째 열 (B 열)을 선택합니다.
if 1 >= len(df.columns):
    raise ValueError("The uploaded Excel file does not have enough columns!")

# FASTA 파일 생성
with open(output_file, "w") as fasta_file:
    for i, seq in enumerate(df[1], start=1): # df[1]을 사용하여 두 번째 열에 접근
        fasta_file.write(f">sequence_{i}\n")
        fasta_file.write(f"{seq}\n")

# 변환 완료 및 다운로드 링크 생성
print(f"FASTA 파일이 생성되었습니다: {output_file}")

# Colab에서 다운로드 제공
files.download(output_file)

Saving tagging.xlsx to tagging (1).xlsx
Uploaded file: tagging (1).xlsx
FASTA 파일이 생성되었습니다: output_sequences.fasta


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from Bio import SeqIO
import re
from google.colab import files

# 1. F 바코드 (F1 ~ F16) 정의
front_barcodes = {
    "F1": "CTTGCTTG", "F2": "GTTGGTTG", "F3": "CATGCATG", "F4": "GATGGATG",
    "F5": "ACTGACTG", "F6": "AGTGAGTG", "F7": "CTAGCTAG", "F8": "GTAGGTAG",
    "F9": "CAAGCAAG", "F10": "GAAGGAAG", "F11": "ACAGACAG", "F12": "AGAGAGAG",
    "F13": "ATCGATCG", "F14": "AACGAACG", "F15": "ATGGATGG", "F16": "AAGGAAGG"
}

# 2. R 바코드 (R1 ~ R24) 정의
rear_barcodes = {
    "R1": "TAGGTAGG", "R2": "TAGCTAGC", "R3": "ATACGTACG", "R4": "TACCTACC",
    "R5": "TTGGTTGG", "R6": "TTGCTTGC", "R7": "TTCGTTCG", "R8": "TTCCTTCC",
    "R9": "GAAGGAAG", "R10": "GAACGAAC", "R11": "GATGGATG", "R12": "GATCGATC",
    "R13": "GAGTGAGT", "R14": "GACTGACT", "R15": "GTAGGTAG", "R16": "GTACGTAC",
    "R17": "GTTGGTTG", "R18": "GTTCGTTC", "R19": "GTGTGTGT", "R20": "GTCTGTCT",
    "R21": "GGATGGAT", "R22": "GGTTGGTT", "R23": "GCATGCAT", "R24": "GCTTGCTT"
}

# 3. F 바코드와 R 바코드 조합으로 이름 생성 (총 384개)
barcode_dict = {}
letters = "ABCDEFGHIJKLMNOP"  # A ~ P (16개의 앞 바코드)
for i, (f_key, f_seq) in enumerate(front_barcodes.items()):  # F1~F16
    for j, (r_key, r_seq) in enumerate(rear_barcodes.items()):  # R1~R24
        barcode_name = f"{letters[i]}{j + 1:02d}"  # A01, A02, ..., P24
        barcode_dict[(f_seq, r_seq)] = barcode_name

# 4. FASTA 파일 업로드
print("FASTA 파일을 업로드하세요.")
uploaded = files.upload()

input_file = list(uploaded.keys())[0]
output_file = "processed_reads.fasta"

# 5. 패턴 정의
pattern1 = "A{10}([ATGC]{8})A{10}"  # 앞 패턴: AAAAAAAAAA + 8개 염기 + AAAAAAAAAA
pattern2 = "T{10}([ATGC]{8})T{10}"  # 뒤 패턴: TTTTTTTTTT + 8개 염기 + TTTTTTTTTT

# 6. 리드 처리
processed_reads = []

with open(input_file, "r") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)

        # 패턴 매칭
        match1 = re.search(pattern1, sequence)
        match2 = re.search(pattern2, sequence)

        if match1 and match2 and match1.end() < match2.start():
            # 7. 바코드 추출
            front_barcode = match1.group(1)  # 앞 바코드 서열
            rear_barcode = match2.group(1)  # 뒤 바코드 서열

            # 8. 리드 이름 매핑
            name = barcode_dict.get((front_barcode, rear_barcode))
            if not name:
                # 매칭되지 않는 조합은 무시
                print(f"Skipping unknown combination: {front_barcode}, {rear_barcode}")
                continue

            # 9. 서열 자르기: 앞 패턴의 끝과 뒤 패턴의 시작 사이
            trimmed_sequence = sequence[match1.end():match2.start()]
            if not trimmed_sequence:
                print(f"Warning: Empty sequence for {record.id}. Skipping...")
                continue

            # 10. 리드 업데이트
            record.id = name
            record.description = ""
            record.seq = trimmed_sequence
            processed_reads.append(record)
        else:
            print(f"Skipping unmatched sequence: {record.id}")

# 11. 결과 저장
with open(output_file, "w") as output_fasta:
    SeqIO.write(processed_reads, output_fasta, "fasta")

print(f"Processed reads saved to {output_file}. Total reads: {len(processed_reads)}")
files.download(output_file)


FASTA 파일을 업로드하세요.


Saving tagging.fasta to tagging (5).fasta
Skipping unknown combination: TCACTCAC, GTGAGTGA
Skipping unknown combination: GATCGATC, GTAGGTAG
Skipping unmatched sequence: sequence_4
Skipping unknown combination: GATCGATC, GTACGTAC
Skipping unknown combination: TGACTACA, ACTCACTC
Skipping unknown combination: GAGTGAGT, GTAGGTAG
Skipping unknown combination: GACTGACT, GTAGGTAG
Skipping unknown combination: TCACTCAC, GATCGATC
Skipping unknown combination: CTACCTAC, GTAGGTAG
Skipping unknown combination: GATCGATC, GTGAGTGA
Skipping unmatched sequence: sequence_12
Skipping unmatched sequence: sequence_13
Skipping unknown combination: GTACGTAC, CTACCTAC
Skipping unmatched sequence: sequence_15
Skipping unmatched sequence: sequence_16
Skipping unknown combination: CTACCTAC, GTAGGTAG
Skipping unknown combination: GACTGACT, GTAGGTAG
Skipping unknown combination: CTACCTAC, GTGAGTGA
Skipping unknown combination: GATCGATC, GTGAGTGA
Skipping unknown combination: TCACTCAC, AGTCAGTC
Skipping unknown co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>